In [1]:
from coffea import processor, hist
import awkward as ak
import numpy as np
import uproot
import numba as nb

from functools import partial
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler

# mpl.use('Agg')
# mpl.rcParams['axes.prop_cycle'] = cycler(color=['blue', 'red', 'green', 'violet', 'darkorange', 'black', 'cyan', 'yellow'])

import boost_histogram as bh
import mplhep as hep

plt.style.use(hep.style.CMS)

from itertools import product
import shutil
import yaml
from types import SimpleNamespace

In [2]:
from coffea.nanoevents import NanoEventsFactory, PFNanoAODSchema
from coffea.nanoevents.methods import vector

events = NanoEventsFactory.from_root(
    "../testfiles/nano_mc2018_ULv2.root",
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "test"},
).events()

dataset = events.metadata["dataset"]
isRealData = not hasattr(events, "genWeight")

# Basic

In [3]:
basic_vars = {
    "Run": events.run,
    "Evt": events.event,
    "LumiBlock": events.luminosityBlock,
    "nPU": events.Pileup.nPU,
    "nPUtrue": events.Pileup.nTrueInt,
    "rho": events.fixedGridRhoFastjetAll,
    "pthat": events.Generator.binvar,
}

# HLT

In [4]:
## HLT


@nb.njit
def to_bitwise_trigger(pass_trig, builder):
    for it in pass_trig:
        # group by every 32 bits
        builder.begin_list()
        for bitidx in range(len(it) // 32 + 1):
            trig = 0
            start = bitidx * 32
            end = min((bitidx + 1) * 32, len(it))
            for i, b in enumerate(it[start:end]):
                trig += b << i
            builder.integer(trig)
        builder.end_list()

    return builder


triggers = [
    "PFJet40",
    "PFJet60",
    "PFJet80",
    "PFJet140",
    "PFJet200",
    "PFJet260",
    "PFJet320",
    "PFJet400",
    "PFJet450",
    "PFJet500",
    "PFJet550",
    "AK8PFJet40",
    "AK8PFJet60",
    "AK8PFJet80",
    "AK8PFJet140",
    "AK8PFJet200",
    "AK8PFJet260",
    "AK8PFJet320",
    "AK8PFJet400",
    "AK8PFJet450",
    "AK8PFJet500",
    "AK8PFJet550",
    "PFHT180",
    "PFHT250",
    "PFHT370",
    "PFHT430",
    "PFHT510",
    "PFHT590",
    "PFHT680",
    "PFHT780",
    "PFHT890",
    "PFHT1050",
    "BTagMu_AK4DiJet20_Mu5",
    "BTagMu_AK4DiJet40_Mu5",
    "BTagMu_AK4DiJet70_Mu5",
    "BTagMu_AK4DiJet110_Mu5",
    "BTagMu_AK4DiJet170_Mu5",
    "BTagMu_AK4Jet300_Mu5",
    "BTagMu_AK8DiJet170_Mu5",
    "BTagMu_AK8Jet300_Mu5",
    "BTagMu_AK4DiJet20_Mu5_noalgo",
    "BTagMu_AK4DiJet40_Mu5_noalgo",
    "BTagMu_AK4DiJet70_Mu5_noalgo",
    "BTagMu_AK4DiJet110_Mu5_noalgo",
    "BTagMu_AK4DiJet170_Mu5_noalgo",
    "BTagMu_AK4Jet300_Mu5_noalgo",
    "BTagMu_AK8DiJet170_Mu5_noalgo",
    "BTagMu_AK8Jet300_Mu5_noalgo",
    "BTagMu_AK8Jet170_DoubleMu5_noalgo",
]
checkHLT = ak.Array([hasattr(events.HLT, _trig) for _trig in triggers])
if ak.all(checkHLT == False):
    raise ValueError("HLT paths:", triggers, " are all invalid in", dataset)
elif ak.any(checkHLT == False):
    print(
        np.array(triggers)[~checkHLT], " does not exist in", dataset, ", please check!"
    )

# pass_trigger dim: (n_evt, n_trig), append False for non-exist triggers
pass_trig = np.array(
    [
        getattr(events.HLT, _trig, np.zeros(len(events), dtype="bool"))
        for _trig in triggers
    ]
).T

# convert to bitwise trigger
# basic_vars['nBitTrigger'] = len(triggers) // 32 + 1
basic_vars["BitTrigger"] = to_bitwise_trigger(pass_trig, ak.ArrayBuilder()).snapshot()

# PV
PV = ak.zip(
    {
        "x": events.PV.x,
        "y": events.PV.y,
        "z": events.PV.z,
        "ndof": events.PV.ndof,
        "chi2": events.PV.chi2,
    }
)

# Quarks

In [5]:
## Quarks


# @nb.njit # sometime has bug... strange!
def is_from_GSP(
    target_idx_list, pdgid_list, mom_idx_list, mom2_idx_list, status_list, builder
):
    # check if the quark is from gluon-splitting
    for target_idx, pdgid, mom_idx, mom2_idx, status in zip(
        target_idx_list, pdgid_list, mom_idx_list, mom2_idx_list, status_list
    ):
        # loop over events

        builder.begin_list()
        for idx in target_idx:
            # loop over target (b/c quark) index

            i = idx
            quark_pdgid = abs(pdgid[idx])
            from_GSP = False
            while (
                mom2_idx[i] == -1
                and mom_idx[i] >= 0
                and (
                    abs(pdgid[mom_idx[i]]) == 21
                    or abs(pdgid[mom_idx[i]]) == quark_pdgid
                )
                and not (
                    abs(pdgid[mom_idx[i]]) == quark_pdgid
                    and status[mom_idx[i]] >= 21
                    and status[mom_idx[i]] <= 29
                )
            ):
                # tracing mother upwards, but the mother cannot be a hard process quark
                if pdgid[mom_idx[i]] == 21:  # is gluon
                    from_GSP = True
                    break
                i = mom_idx[i]
            builder.boolean(from_GSP)

        builder.end_list()
    return builder


for quark_pdgid in [4, 5]:
    # finding b or c quarks

    quark_sel = (abs(events.GenPart.pdgId) == quark_pdgid) & (
        (events.GenPart.statusFlags & (1 << 13)) > 0
    )  # is b/c-quark & is last copy
    quark_index = ak.local_index(events.GenPart.pdgId)[quark_sel]
    quarks = events.GenPart[quark_sel]
    out = ak.zip(
        {
            "pT": quarks.pt,
            "eta": quarks.eta,
            "phi": quarks.phi,
            "pdgId": quarks.pdgId,
            "status": quarks.status,
            "fromGSP": ak.values_astype(
                is_from_GSP(
                    quark_index,
                    events.GenPart.pdgId,
                    events.GenPart.genPartIdxMother,
                    events.GenPart.genPartIdxMother2,
                    events.GenPart.status,
                    ak.ArrayBuilder(),
                ).snapshot(),
                int,
            ),
        }
    )
    if quark_pdgid == 4:
        cQuark = out
    elif quark_pdgid == 5:
        bQuark = out

# Hadrons

In [14]:
## Hadrons

# mass table from https://github.com/scikit-hep/particle/blob/master/src/particle/data/particle2022.csv
hadron_mass_table = {
    411: 1.86966,
    413: 2.01026,
    415: 2.4611,
    421: 1.86484,
    423: 2.00685,
    425: 2.4611,
    431: 1.96835,
    433: 2.1122,
    435: 2.5691,
    441: 2.9839,
    443: 3.0969,
    445: 3.55617,
    511: 5.27966,
    513: 5.32471,
    515: 5.7395,
    521: 5.27934,
    523: 5.32471,
    525: 5.7372,
    531: 5.36692,
    533: 5.4154,
    535: 5.83986,
    541: 6.27447,
    553: 9.4603,
    555: 9.9122,
    4101: -0.001,
    4103: -0.001,
    4112: 2.45375,
    4114: 2.51848,
    4122: 2.28646,
    4132: 2.47044,
    4201: -0.001,
    4203: -0.001,
    4212: 2.45265,
    4214: 2.5174,
    4222: 2.45397,
    4224: 2.51841,
    4232: 2.46771,
    4301: -0.001,
    4303: -0.001,
    4312: 2.5787,
    4314: 2.64616,
    4322: 2.5782,
    4324: 2.6451,
    4332: 2.6952,
    4334: 2.7659,
    4403: -0.001,
    5101: -0.001,
    5103: -0.001,
    5112: 5.81564,
    5114: 5.83474,
    5122: 5.6196,
    5132: 5.797,
    5201: -0.001,
    5203: -0.001,
    5222: 5.81056,
    5224: 5.83032,
    5232: 5.7919,
    5301: -0.001,
    5303: -0.001,
    5332: 6.0452,
    5401: -0.001,
    5403: -0.001,
    5503: -0.001,
}
hadron_pids = ak.Array(list(hadron_mass_table.keys()))
hadron_masses = ak.Array(list(hadron_mass_table.values()))


@nb.njit
def get_hadron_mass(pids_list, hadron_pids, hadron_masses, builder):
    for pids in pids_list:
        # loop over events
        builder.begin_list()
        for p in pids:
            for i in range(len(hadron_pids)):
                if hadron_pids[i] == abs(p):
                    builder.real(hadron_masses[i])
                    break
            else:
                builder.real(-1)
        builder.end_list()
    return builder


is_heavy_hadron = lambda p, pid: (abs(p.pdgId) // 100 == pid) | (
    abs(p.pdgId) // 1000 == pid
)

# finding D hadrons
sel = is_heavy_hadron(events.GenPart, 4) & (
    (events.GenPart.statusFlags & (1 << 13)) > 0
)  # PID match, is last copy
chadrons = events.GenPart[sel]
chadrons = chadrons[
    ~ak.any(is_heavy_hadron(chadrons.children, 4), axis=-1)
]  # should not select D hadron whose daughters also includes D hadrons
DHadron = ak.zip(
    {
        "pT": chadrons.pt,
        "eta": chadrons.eta,
        "phi": chadrons.phi,
        "pdgID": chadrons.pdgId,
        "mass": get_hadron_mass(
            chadrons.pdgId, hadron_pids, hadron_masses, ak.ArrayBuilder()
        ).snapshot(),
        "muInDaughter": ak.any(abs(chadrons.children.pdgId) == 13, axis=-1),
    }
)

# finding B hadrons
sel = is_heavy_hadron(events.GenPart, 5) & (
    (events.GenPart.statusFlags & (1 << 13)) > 0
)  # PID match, is last copy
bhadrons = events.GenPart[sel]
BHadron = ak.zip(
    {
        "pT": bhadrons.pt,
        "eta": bhadrons.eta,
        "phi": bhadrons.phi,
        "pdgID": bhadrons.pdgId,
        "mass": get_hadron_mass(
            bhadrons.pdgId, hadron_pids, hadron_masses, ak.ArrayBuilder()
        ).snapshot(),
        "hasBdaughter": ak.values_astype(
            ak.any(is_heavy_hadron(bhadrons.children, 5), axis=-1), int
        ),  # B hadrons with B-daughters not removed
    }
)

## find all daughter stable D hadrons and matched with D hadron list
bhad_ch1G = bhadrons.children
bhad_ch2G = bhadrons.children.children
bhad_ch3G = bhadrons.children.children.children
bhad_ch4G = bhadrons.children.children.children.children
# for each B hadron: form a list of it daugthers which are:
# D hadron & D does not include D hadron daugthers
# i.e., find B -> D
chad_asdau_1G = bhad_ch1G[
    is_heavy_hadron(bhad_ch1G, 4) & ~ak.any(is_heavy_hadron(bhad_ch2G, 4), axis=-1)
]
# then find B -> D* -> D (store the final D)
chad_asdau_2G = ak.flatten(
    bhad_ch2G[
        is_heavy_hadron(bhad_ch2G, 4) & ~ak.any(is_heavy_hadron(bhad_ch3G, 4), axis=-1)
    ],
    axis=3,
)
# then find B -> D** -> D* -> D (store the final D)
chad_asdau_3G = ak.flatten(
    ak.flatten(
        bhad_ch3G[
            is_heavy_hadron(bhad_ch3G, 4)
            & ~ak.any(is_heavy_hadron(bhad_ch4G, 4), axis=-1)
        ],
        axis=4,
    ),
    axis=3,
)
# combine all final D hadrons together
# now in dim: (evt, bhad, chad_asdau)
chad_asdau_final = ak.concatenate([chad_asdau_1G, chad_asdau_2G, chad_asdau_3G], axis=2)

chadrons_broadcast = ak.cartesian([bhadrons, chadrons], axis=1, nested=True)[
    "1"
]  # broadcast to include bhad dimension: (evt, bhad, chad)

# pair the D hadrons (as daughters) with the previous D hadron list
pair = ak.cartesian(
    [chad_asdau_final, chadrons_broadcast], axis=2, nested=True
)  # dim: (evt, bhad, chad_asdau, chad)
chad_index = ak.argmax(
    pair["0"].pt == pair["1"].pt, axis=-1
)  # dim: (evt, bhad, chad_asdau)

# fill D hadron index (exclude those with hasBdaughter==False)
BHadron["DHadron1"] = ak.fill_none(
    ak.where(
        (ak.num(chad_index, axis=-1) > 0) & ~BHadron.hasBdaughter,
        ak.pad_none(chad_index, 2, axis=2)[:, :, 0],
        ak.zeros_like(BHadron.pT, dtype=int) - 1,
    ),
    -1,
)
BHadron["DHadron2"] = ak.fill_none(
    ak.where(
        (ak.num(chad_index, axis=-1) > 1) & ~BHadron.hasBdaughter,
        ak.pad_none(chad_index, 2, axis=2)[:, :, 1],
        ak.zeros_like(BHadron.pT, dtype=int) - 1,
    ),
    -1,
)

# Genlep & V0

In [15]:
## Genlep

_fix = lambda x: ak.fill_none(x, 0)
is_lep = (
    lambda p: (abs(_fix(p.pdgId)) == 11)
    | (abs(_fix(p.pdgId)) == 13)
    | (abs(_fix(p.pdgId)) == 15)
)
is_WZ = lambda p: (abs(_fix(p.pdgId)) == 23) | (abs(_fix(p.pdgId)) == 24)
is_heavy_hadron = lambda p, pid: (abs(_fix(p.pdgId)) // 100 == pid) | (
    abs(_fix(p.pdgId)) // 1000 == pid
)

sel = (
    is_lep(events.GenPart)
    & ((events.GenPart.statusFlags & (1 << 13)) > 0)
    & (events.GenPart.pt > 3.0)
)  # requires pT > 3 GeV
genlep = events.GenPart[sel]

# trace parents up to 4 generations (from BTA code)
genlep_pa1G = genlep.parent
genlep_pa2G = genlep.parent.parent
genlep_pa3G = genlep.parent.parent.parent
genlep_pa4G = genlep.parent.parent.parent.parent
istau = abs(genlep_pa1G.pdgId) == 15
isWZ = is_WZ(genlep_pa1G) | is_WZ(genlep_pa2G)
isD = is_heavy_hadron(genlep_pa1G, 4) | is_heavy_hadron(genlep_pa2G, 4)
isB = (
    is_heavy_hadron(genlep_pa1G, 5)
    | is_heavy_hadron(genlep_pa2G, 5)
    | is_heavy_hadron(genlep_pa3G, 5)
    | is_heavy_hadron(genlep_pa4G, 5)
)

Genlep = ak.zip(
    {
        "pT": genlep.pt,
        "eta": genlep.eta,
        "phi": genlep.phi,
        "pdgID": genlep.pdgId,
        "status": genlep.status,
        "mother": ak.fill_none(
            ak.where(isB | isD, 5 * isB + 4 * isD, 10 * istau + 100 * isWZ), 0
        ),
    }
)

# V0

is_V0 = lambda p: (abs(p.pdgId) == 310) | (abs(p.pdgId) == 3122)
sel = is_V0(events.GenPart) & ((events.GenPart.statusFlags & (1 << 13)) > 0)
genV0 = events.GenPart[sel]

# finding charged daughters with pT > 1
is_lep = (
    lambda p: (abs(_fix(p.pdgId)) == 11)
    | (abs(_fix(p.pdgId)) == 13)
    | (abs(_fix(p.pdgId)) == 15)
)
is_charged_light_hadron = (
    lambda p: (abs(p.pdgId) == 211)
    | (abs(p.pdgId) == 213)
    | (abs(p.pdgId) == 321)
    | (abs(p.pdgId) == 323)
)
genV0_ch_charged = genV0.children[
    (is_charged_light_hadron(genV0.children) | is_lep(genV0.children))
    & (genV0.children.pt > 1.0)
]
ncharged = ak.num(genV0_ch_charged, axis=2)

# check acceptance (from the BTA code)
svx = ak.fill_none(ak.firsts(genV0_ch_charged.vx, axis=-1), 0)
svy = ak.fill_none(ak.firsts(genV0_ch_charged.vy, axis=-1), 0)
svz = ak.fill_none(ak.firsts(genV0_ch_charged.vz, axis=-1), 0)
radius = np.hypot(svx, svy)
abseta = abs(genV0.eta)

in_acceptance = (ncharged > 0) & (
    ((abseta < 2.0) & (radius < 7.3))
    | ((abseta < 2.5) & (radius < 4.4))
    | ((abseta > 1.8) & (abseta < 2.5) & (abs(svz) < 34.5))
)

genV0_inaccept = genV0[in_acceptance]
GenV0 = ak.zip(
    {
        "pT": genV0_inaccept.pt,
        "eta": genV0_inaccept.eta,
        "phi": genV0_inaccept.phi,
        "pdgId": genV0_inaccept.pdgId,
        "nCharged": ncharged[in_acceptance],
        "SVx": svx[in_acceptance],
        "SVy": svy[in_acceptance],
        "SVz": svz[in_acceptance],
    }
)

# Jets

In [16]:
# Jets

jet = events.Jet[
    (events.Jet.pt > 20.0) & (abs(events.Jet.eta) < 2.5)
]  # basic selection
zeros = ak.zeros_like(jet.pt, dtype=int)
Jet = ak.zip(
    {
        # basic kinematics
        "pt": jet.pt,
        "eta": jet.eta,
        "phi": jet.phi,
        "mass": jet.mass,
        "uncorrpt": (1 - jet.rawFactor) * jet.pt,
        # jet ID/pileup ID // !!!
        "looseID": ak.values_astype(
            zeros + 1, int
        ),  # jet.jetId & (1 << 0) > 0, # has some problem..? always 0
        "tightID": ak.values_astype(jet.jetId & (1 << 1) > 0, int),
        # 'tightlepvetoID': jet.jetId & (1 << 2) > 0,
        # pileup ID (essentially userInt('puId106XUL18Id') for UL18)
        "pileup_tightID": ak.values_astype(
            (jet.puId & (1 << 0) > 0) | (jet.pt > 50.0), int
        ),
        "pileup_mediumID": ak.values_astype(
            (jet.puId & (1 << 1) > 0) | (jet.pt > 50.0), int
        ),
        "pileup_looseID": ak.values_astype(
            (jet.puId & (1 << 2) > 0) | (jet.pt > 50.0), int
        ),
        # taggers/vars // JP/JBP to be calibrated.. !!!
        "area": jet.area,
        "Proba": jet.Proba,
        "ProbaN": jet.ProbaN,
        "Bprob": jet.Bprob,
        "BprobN": jet.BprobN,
        # taggers
        "DeepFlavourBDisc": jet.btagDeepFlavB,
        "DeepFlavourCvsLDisc": jet.btagDeepFlavCvL,
        "DeepFlavourCvsBDisc": jet.btagDeepFlavCvB,
        "DeepFlavourBDiscN": jet.btagNegativeDeepFlavB,
        "DeepFlavourCvsLDiscN": jet.btagNegativeDeepFlavCvL,
        "DeepFlavourCvsBDiscN": jet.btagNegativeDeepFlavCvB,
    }
)

if not isRealData:
    Jet["nbHadrons"] = jet.nBHadrons
    Jet["ncHadrons"] = jet.nCHadrons
    Jet["partonFlavour"] = jet.partonFlavour
    Jet["hadronFlavour"] = jet.hadronFlavour
    # corrected flavours defined in BTA
    Jet["flavour"] = ak.where(
        jet.hadronFlavour != 0,
        jet.hadronFlavour,
        ak.where(
            (abs(jet.partonFlavour) == 4) | (abs(jet.partonFlavour) == 5),
            zeros,
            jet.partonFlavour,
        ),
    )

    # genJet pT
    genJetIdx = ak.where(
        jet.genJetIdx < ak.num(events.GenJet), jet.genJetIdx, zeros - 1
    )  # in case the genJet index out of range
    Jet["genpt"] = ak.where(genJetIdx >= 0, events.GenJet[genJetIdx].pt, zeros - 1)

    # gen-level jet cleaning aginst prompt leptons
    genlep_prompt = genlep[(Genlep.mother != 0) & (Genlep.mother % 10 == 0)]
    overlapped = ak.zeros_like(jet.pt, dtype=bool)

    for lep_pid, dr_thres in [(11, 0.2), (13, 0.2), (15, 0.3)]:
        lep = genlep_prompt[abs(genlep_prompt.pdgId) == lep_pid]
        pair = ak.cartesian([jet, lep], axis=1, nested=True)  # dim: (event, jet, lep)
        dr_min = ak.min(pair["0"].delta_r(pair["1"]), axis=-1)  # dim: (event, jet)
        dr_min = ak.fill_none(dr_min, 99.0)  # for 0 lepton case, assign dr=99
        overlapped = overlapped | (dr_min < dr_thres)
    Jet["flavourCleaned"] = ak.where(
        overlapped, zeros - 999, Jet["flavour"]
    )  # fill -999 for jets with lepton overlapping

    # jet cleaning aginst pileup
    Jet["flavourCleaned"] = ak.where(Jet["genpt"] < 8.0, zeros, Jet["flavourCleaned"])


# CSV inputs per jets: retreived from the DeepCSV input given they are the same
TagVarCSV = ak.zip(
    {
        "trackJetPt": jet.DeepCSV_trackJetPt,
        "vertexCategory": jet.DeepCSV_vertexCategory,
        "jetNSecondaryVertices": jet.DeepCSV_jetNSecondaryVertices,
        "trackSumJetEtRatio": jet.DeepCSV_trackSumJetEtRatio,
        "trackSumJetDeltaR": jet.DeepCSV_trackSumJetDeltaR,
        "vertexMass": jet.DeepCSV_vertexMass,
        "vertexNTracks": jet.DeepCSV_vertexNTracks,
        "vertexEnergyRatio": jet.DeepCSV_vertexEnergyRatio,
        "vertexJetDeltaR": jet.DeepCSV_vertexJetDeltaR,
    }
)

# TrkInc

In [17]:
@nb.njit
def cumsum(array, builder):
    for arr in array:
        builder.begin_list()
        sum = 0
        for i in arr:
            sum += i
            builder.integer(sum)
        builder.end_list()
    return builder


# # match jets with PFCands
# trk = events.PFCands[(events.PFCands.trkQuality != 0) & (events.PFCands.pt > 5.)]
# pair = ak.cartesian([jet, trk], axis=1, nested=True) # dim: (event, jet, pfcand)
# matched = pair['0'].delta_r(pair['1']) < 0.4
# trk_jetmatch = pair['1'][matched]

## match with JetPFCands
# form the track selection from JetPFCands
trkj = events.JetPFCands[
    (events.JetPFCands.pf.trkQuality != 0) & (events.JetPFCands.pt > 1.0)
]

# selection for "TrkInc" in BTA
trkj = trkj[
    (trkj.pf.trkHighPurity == 1)
    & (trkj.pf.trkAlgo != 9)
    & (trkj.pf.trkAlgo != 10)
    & (trkj.pt > 5.0)
    & (trkj.pf.numberOfHits >= 11)
    & (trkj.pf.numberOfPixelHits >= 2)
    & (trkj.pf.trkChi2 < 10)
    & (trkj.pf.lostOuterHits <= 2)
    & (trkj.pf.dz < 1.0)
]

pair = ak.cartesian([jet, trkj], axis=1, nested=True)  # dim: (event, jet, pfcand)
# matched = pair['0'].pt == pair['1'].jet.pt
matched = (pair["0"].pt == pair["1"].jet.pt) & (pair["0"].delta_r(pair["1"].pf) < 0.4)
trkj_jetbased = pair["1"][matched]  # dim: (event, jet, pfcand)

# calculate pTrel
vec = ak.zip(
    {
        "pt": trkj_jetbased.pf.trkPt,
        "eta": trkj_jetbased.pf.trkEta,
        "phi": trkj_jetbased.pf.trkPhi,
        "mass": trkj_jetbased.pf.mass,
    },
    behavior=vector.behavior,
    with_name="PtEtaPhiMLorentzVector",
)
ptperp = vec.dot(jet) / jet.p
trkj_jetbased["ptrel"] = np.sqrt(np.maximum(vec.p**2 - ptperp**2, 0))

# flatten jet-based track arrays
trkj_jetbased_flat = ak.flatten(trkj_jetbased, axis=2)  # dim: (event, pfcand)
trkj_jetbased_num = ak.num(trkj_jetbased, axis=2)

TrkInc = ak.zip(
    {
        "pt": ak.fill_none(trkj_jetbased_flat.pf.trkPt, 0),
        "eta": ak.fill_none(trkj_jetbased_flat.pf.trkEta, 0),
        "phi": ak.fill_none(trkj_jetbased_flat.pf.trkPhi, 0),
        "ptrel": ak.fill_none(trkj_jetbased_flat.ptrel, 0),
    }
)

# assign the first and last index of matched muons to Jet
lastidx = cumsum(trkj_jetbased_num, ak.ArrayBuilder()).snapshot()
firstidx = ak.fill_none(
    ak.concatenate(
        [
            ak.singletons(ak.zeros_like(events.run, dtype=int)),
            lastidx.mask[ak.num(lastidx) > 0][:, :-1],
        ],
        axis=1,
    ),
    0,
)
Jet["nFirstTrkInc"] = firstidx
Jet["nLastTrkInc"] = lastidx

# PFMuon

In [18]:
# PFMuon

# find muon PFCands inside a jet
# based on SoftPFMuonTagInfoProducer:
# https://github.com/cms-sw/cmssw/blob/10_6_X/RecoBTag/SoftLepton/plugins/SoftPFMuonTagInfoProducer.cc

mutrkj = events.JetPFCands[
    (events.JetPFCands.pf.trkQuality != 0)
    & (events.JetPFCands.pt > 2.0)
    & (abs(events.JetPFCands.pf.pdgId) == 13)
]

# match muon PFCands to muons
pair = ak.cartesian(
    [mutrkj, events.Muon], axis=1, nested=True
)  # dim: (event, pfcand, mu)
matched = (
    (pair["0"].pf.pdgId == pair["1"].pdgId)
    & (pair["0"].pf.delta_r(pair["1"]) < 0.01)
    & ((pair["0"].pt - pair["1"].pt) / pair["0"].pt < 0.1)
)
mutrkj["mu"] = ak.firsts(
    pair["1"][matched], axis=2
)  # dim same with mutrkj: (event, matched-pfcand), None if not matching with a muon

# match muon PFCands also to jets
pair = ak.cartesian([jet, mutrkj], axis=1, nested=True)  # dim: (event, jet, pfcand)
matched = pair["0"].pt == pair["1"].jet.pt
mutrkj_jetbased = pair["1"][matched]  # dim: (event, jet, matched-pfcand)

# further requirement: must match with a muon and require the loose ID
mutrkj_jetbased = mutrkj_jetbased[
    ak.fill_none(mutrkj_jetbased.mu.looseId, False)
]  # dim: (event, jet, matched-muons)
mu_jetbased = mutrkj_jetbased.mu

# calculate pTrel and other kinematics
ptperp = mu_jetbased.dot(jet) / jet.p
mu_jetbased["ptrel"] = np.sqrt(mu_jetbased.p**2 - ptperp**2)
# ratio and ratioRel seems different from the BTA value..
mu_jetbased["ratio"] = mu_jetbased.pt / jet.pt
mu_jetbased["ratioRel"] = (mu_jetbased.t * jet.t - mu_jetbased.dot(jet)) / jet.mass2

mu_jetbased["deltaR"] = mu_jetbased.delta_r(jet)

# quality
zeros = ak.zeros_like(mu_jetbased.pt, dtype=int)
mu_jetbased["GoodQuality"] = zeros
mu_jetbased["GoodQuality"] = ak.where(
    mu_jetbased.isGlobal, zeros + 1, mu_jetbased["GoodQuality"]
)
mu_jetbased["GoodQuality"] = ak.where(
    (mu_jetbased["GoodQuality"] == 1)
    & (mu_jetbased.nStations >= 2)
    & (mutrkj_jetbased.pf.numberOfHits >= 11)
    & (mutrkj_jetbased.pf.numberOfPixelHits >= 2)
    & (mutrkj_jetbased.pf.trkChi2 < 10),
    zeros + 2,
    mu_jetbased["GoodQuality"],
)

# flatten jet-based track arrays
mu_jetbased_flat = ak.flatten(mu_jetbased, axis=2)  # dim: (event, pfcand)
mu_jetbased_jetidx_flat = ak.flatten(
    ak.broadcast_arrays(ak.local_index(jet, axis=1), mu_jetbased.pt)[0], axis=2
)
mu_jetbased_num = ak.num(mu_jetbased, axis=2)

PFMuon = ak.zip(
    {
        "IdxJet": ak.fill_none(mu_jetbased_jetidx_flat, 0),
        "pt": ak.fill_none(mu_jetbased_flat.pt, 0),
        "eta": ak.fill_none(mu_jetbased_flat.eta, 0),
        "phi": ak.fill_none(mu_jetbased_flat.phi, 0),
        "ptrel": ak.fill_none(mu_jetbased_flat.ptrel, 0),
        "ratio": ak.fill_none(mu_jetbased_flat.ratio, 0),
        "ratioRel": ak.fill_none(mu_jetbased_flat.ratioRel, 0),
        "deltaR": ak.fill_none(mu_jetbased_flat.deltaR, 0),
        "IP": ak.fill_none(
            mu_jetbased_flat.ip3d * np.sign(mu_jetbased_flat.dxy + mu_jetbased_flat.dz),
            0,
        ),
        "IPsig": ak.fill_none(
            mu_jetbased_flat.sip3d
            * np.sign(mu_jetbased_flat.dxy + mu_jetbased_flat.dz),
            0,
        ),
        "IP2D": ak.fill_none(mu_jetbased_flat.dxy, 0),
        "IP2Dsig": ak.fill_none(mu_jetbased_flat.dxy / mu_jetbased_flat.dxyErr, 0),
        "GoodQuality": ak.fill_none(mu_jetbased_flat.GoodQuality, 0),
    }
)

# assign the first and last index of matched muons to Jet
lastidx = cumsum(mu_jetbased_num, ak.ArrayBuilder()).snapshot()
firstidx = ak.fill_none(
    ak.concatenate(
        [
            ak.singletons(ak.zeros_like(events.run, dtype=int)),
            lastidx.mask[ak.num(lastidx) > 0][:, :-1],
        ],
        axis=1,
    ),
    0,
)
Jet["nFirstSM"] = firstidx
Jet["nLastSM"] = lastidx

# Write to ROOT

In [19]:
output = {
    **basic_vars,
    "PV": PV,
    "bQuark": bQuark,
    "cQuark": cQuark,
    "BHadron": BHadron,
    "DHadron": DHadron,
    "Genlep": Genlep,
    "GenV0": GenV0,
    "Jet": Jet,
    "TagVarCSV": TagVarCSV,
    "TrkInc": TrkInc,
    "PFMuon": PFMuon,
}
for key in output:
    if key in basic_vars:
        print(key, output[key])
    else:
        for b in ak.fields(output[key]):
            print(key, b, output[key][b])

Run [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Evt [112068, 112135, 112128, 112124, 112073, ... 114279, 114255, 114237, 114267, 114268]
LumiBlock [13, 13, 13, 13, 13, 13, 13, 13, 13, 13, ... 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
nPU [19, 34, 27, 28, 22, 30, 24, 56, 29, 13, ... 38, 25, 34, 24, 46, 12, 37, 42, 35, 37]
nPUtrue [19, 39, 34, 27, 13, 30, 23, 48, 27, 19, ... 38, 20, 37, 32, 41, 14, 33, 28, 32, 42]
rho [9.18, 24.3, 21.8, 18.4, 20.8, 22.2, 12.9, ... 30.3, 14.3, 29.1, 20.8, 19.5, 25.9]
pthat [550, 595, 569, 542, 485, 493, 511, 585, ... 494, 476, 485, 516, 500, 592, 559, 496]
BitTrigger [[2144335999, 57311], [4294967295, 65535, ... 65280], [2144336383, 65535]]
PV x [0.0107, 0.0105, 0.0112, 0.011, 0.0107, ... 0.00977, 0.0102, 0.0112, 0.0107, 0.0101]
PV y [0.0411, 0.0411, 0.0418, 0.0407, 0.0406, ... 0.0408, 0.0418, 0.0422, 0.0424, 0.0415]
PV z [-6.12, 2.12, -3.42, 1.9, -3.55, 2.33, ... -0.554, -1.02, 1.76, 0.541, 4.99, 2.54]
PV ndof [106

In [19]:
with uproot.recreate("btatuple_qcdmu_v3.5.root") as f:
    f["btagana/ttree"] = output

In [ ]:
######################### END ##########################

# Test

In [49]:
lt = events.JetPFCands[5]
for i in range(len(lt.pt)):
    for b in ["pt", "eta", "phi"]:
        print(b, lt.pf[b][i])
    print("jetIdx", lt.jetIdx[i])

pt 2.484375
eta -1.56640625
phi 2.87451171875
jetIdx 0
pt 5.5234375
eta -1.44140625
phi 3.00048828125
jetIdx 0
pt 2.255859375
eta -1.40380859375
phi 3.10791015625
jetIdx 0
pt 1.552734375
eta -1.545166015625
phi -2.861328125
jetIdx 0
pt 2.498046875
eta -1.631591796875
phi -3.08154296875
jetIdx 0
pt 8.7265625
eta -1.634521484375
phi 3.13720703125
jetIdx 0
pt 215.375
eta -1.60693359375
phi 3.05224609375
jetIdx 0
pt 0.30224609375
eta -1.8349609375
phi 3.03515625
jetIdx 0
pt 0.495849609375
eta -1.5908203125
phi 3.06982421875
jetIdx 0
pt 0.235107421875
eta -1.56005859375
phi 2.95556640625
jetIdx 0
pt 0.286376953125
eta -1.29541015625
phi -2.99658203125
jetIdx 0
pt 4.375
eta -1.3486328125
phi 3.05859375
jetIdx 0
pt 1.7099609375
eta -1.354736328125
phi 2.7451171875
jetIdx 0
pt 0.2247314453125
eta -1.430908203125
phi -3.013671875
jetIdx 0
pt 1.134765625
eta -1.43115234375
phi 3.07958984375
jetIdx 0
pt 0.28466796875
eta -1.431640625
phi 3.0107421875
jetIdx 0
pt 5.41015625
eta -1.5166015625
phi 3


eta 1.798095703125
phi 0.0646514892578125
jetIdx 1
pt 3.822265625
eta 1.79296875
phi 0.1066436767578125
jetIdx 1
pt 5.85546875
eta 1.7919921875
phi 0.115234375
jetIdx 1
pt 15.46875
eta 1.76806640625
phi 0.1193389892578125
jetIdx 1
pt 2.732421875
eta 1.767822265625
phi -0.165435791015625
jetIdx 1
pt 26.40625
eta 1.737060546875
phi 0.0631866455078125
jetIdx 1
pt 13.2421875
eta 1.736083984375
phi 0.0935516357421875
jetIdx 1
pt 11.7890625
eta 1.73486328125
phi 0.0758819580078125
jetIdx 1
pt 14.8984375
eta 1.71435546875
phi 0.05381011962890625
jetIdx 1
pt 2.2734375
eta 1.710205078125
phi -0.0666961669921875
jetIdx 1
pt 22.546875
eta 1.701904296875
phi 0.0777435302734375
jetIdx 1
pt 1.5146484375
eta 1.662353515625
phi -0.008991241455078125
jetIdx 1
pt 2.53515625
eta 2.0185546875
phi -0.063873291015625
jetIdx 1
pt 1.736328125
eta 1.797119140625
phi 0.25860595703125
jetIdx 1
pt 8.09375
eta 1.794677734375
phi 0.1025390625
jetIdx 1
pt 1.03515625
eta 1.996337890625
phi 0.09454345703125
jetIdx 1


In [36]:
ie = 5
for i, (start, end) in enumerate(zip(Jet.nFirstTrkInc[ie], Jet.nLastTrkInc[ie])):
    print(i, "------")
    for b in ak.fields(TrkInc):
        print(b, TrkInc[b][ie][start:end])

0 ------
pt [8.73, 215]
eta [-1.63, -1.61]
phi [3.14, 3.05]
ptrel [0.846, 2.06]
1 ------
pt [24.2, 29, 50.4, 5.86, 15.5, 26.4, 13.2, 11.8, 14.9, 22.5, 8.09, 7.82]
eta [1.82, 1.81, 1.8, 1.79, 1.77, 1.74, 1.74, 1.74, 1.71, 1.7, 1.79, 1.89]
phi [0.0702, 0.0596, 0.0646, 0.115, 0.119, ... 0.0759, 0.0538, 0.0778, 0.103, 0.0411]
ptrel [1.01, 0.977, 0.953, 0.291, 0.813, 1.09, 0.652, 0.517, 0.966, 1.69, 0.312, 0.96]
2 ------
pt []
eta []
phi []
ptrel []
3 ------
pt []
eta []
phi []
ptrel []


In [ ]:
# for ie in range(20):
ie = 12
print(ie, "----------------")
for i, (start, end) in enumerate(zip(Jet.nFirstSM[ie], Jet.nLastSM[ie])):
    print("jet", i, "------")
    for b in ak.fields(PFMuon):
        print(b, PFMuon[b][ie][start:end])

In [ ]:
dir(mu_jetbased)

In [2]:
df = uproot.lazy("btatuple_qcdmu_v3.root")

In [4]:
df[2].PFMuon_pt

<Array [6.76, 5.25] type='2 * float64'>

In [ ]:
for ie in range(len(events)):
    print(ie, "--------------------")
    for b in ak.fields(GenV0):
        print(b, GenV0[b][ie])

# Test with BTA

In [21]:
df = uproot.lazy("../testfiles/JetTree_mc.root:btagana/ttree")

In [ ]:
# for ie in range(len(events)):
#     if len(bQuark[ie]) != len(df[ie].bQuark_pT) or (len(bQuark[ie]) > 0 and (sum(abs(bQuark[ie].pT - df[ie].bQuark_pT)) > 2 or sum(bQuark[ie].fromGSP != ak.values_astype(df[ie].bQuark_fromGSP, 'bool')) > 0)):
#         print(ie)

# for ie in range(len(events)):
#     if len(cQuark[ie]) != len(df[ie].cQuark_pT) or (len(cQuark[ie]) > 0 and (sum(abs(cQuark[ie].pT - df[ie].cQuark_pT)) > 2 or sum(cQuark[ie].fromGSP != ak.values_astype(df[ie].cQuark_fromGSP, 'bool')) > 0)):
#         print(ie)


# for ie in range(100):
#     if len(DHadron[ie]) != len(df[ie].DHadron_pT) or (len(DHadron[ie]) > 0 and (sum(abs(DHadron[ie].pT - df[ie].DHadron_pT)) > 2)):
#         print(ie)

# for ie in range(200):
#     if len(Genlep[ie]) != len(df[ie].Genlep_pT) or (len(Genlep[ie]) > 0 and (sum(abs(Genlep[ie].pT - df[ie].Genlep_pT)) > 2) and (sum(abs(Genlep[ie].mother - df[ie].Genlep_mother)) > 0)):
#         print(ie, len(Genlep[ie]), len(df[ie].Genlep_pT), sum(abs(Genlep[ie].pT - df[ie].Genlep_pT)) > 2, sum(abs(Genlep[ie].mother - df[ie].Genlep_mother)) > 0)

# for ie in range(100):
#     if len(GenV0[ie]) != len(df[ie].GenV0_pT) or (len(GenV0[ie]) > 0 and (sum(abs(GenV0[ie].pT - df[ie].GenV0_pT)) > 2) and (sum(abs(GenV0[ie].SVx - df[ie].GenV0_SVx)) > 0)):
#         print(ie)


# # GenV0
# for ie in range(200):
#     if len(Genlep[ie]) != len(df[ie].Genlep_pT):
#         print(ie, 'length doesnt match', len(Genlep[ie]), len(df[ie].Genlep_pT))
#     else:
#         for b in ak.fields(Genlep):
#             if len(Genlep[ie]) > 0 and any(abs(Genlep[ie][b] - df[ie]['Genlep_'+b]) / (df[ie]['Genlep_'+b]) > 1e-2):
#                 print(ie, b, 'doesnt match', Genlep[ie][b], df[ie]['Genlep_'+b])

# Jets
for ie in range(100):
    if len(Jet[ie]) != len(df[ie].Jet_pt):
        print(ie, "length doesnt match", len(Jet[ie]), len(df[ie].Jet_pt))
    else:
        for b in ak.fields(Jet):
            if len(Jet[ie]) > 0 and any(
                abs(Jet[ie][b] - df[ie]["Jet_" + b]) / (df[ie]["Jet_" + b] + 1e-3)
                > 1e-3
            ):
                if "Prob" not in b and "Bprob" not in b:
                    print(ie, b, "doesnt match", Jet[ie][b], df[ie]["Jet_" + b])

In [ ]:
for ie in range(len(events)):
    for var in basic_vars:
        if abs(basic_vars[var][ie] - df[ie][var]) / (df[ie][var] + 1e-3) > 1e-3:
            print(ie, var, "doesnt match", basic_vars[var][ie], df[ie][var])

In [35]:
for ie in range(100):
    if len(TrkInc[ie]) != len(df[ie].TrkInc_pt):
        print(ie, "length doesnt match", len(TrkInc[ie]), len(df[ie].TrkInc_pt))
    else:
        for b in ak.fields(TrkInc):
            if len(TrkInc[ie]) > 0 and any(
                abs(TrkInc[ie][b] - df[ie]["TrkInc_" + b])
                / (df[ie]["TrkInc_" + b] + 1e-3)
                > 1e-3
            ):
                print(ie, b, "doesnt match", TrkInc[ie][b], df[ie]["TrkInc_" + b])

0 ptrel doesnt match [0.608, 0.652, 0.497, 0.319, 0.425, 0.78, ... 0.203, 0.226, 0.419, 0.372, 0.385] [0.609, 0.655, 0.511, 0.324, 0.421, 0.784, ... 0.201, 0.226, 0.419, 0.371, 0.388]
1 ptrel doesnt match [1.67, 10.6, 0.642, 0.61, 1.23, 1.96, 0.289, ... 3.9, 0.83, 0.827, 0.591, 0.56, 1.73] [1.66, 10.6, 0.642, 0.606, 1.24, 1.96, ... 3.95, 0.819, 0.823, 0.589, 0.559, 1.73]
2 length doesnt match 18 17
3 ptrel doesnt match [0.944, 0.095, 0.041, 0.188, 0.288, 0.611, ... 0.497, 0.136, 1.51, 0.359, 0.544] [0.952, 0.0975, 0.0396, 0.211, 0.275, 0.606, ... 0.498, 0.129, 1.51, 0.358, 0.544]
4 ptrel doesnt match [0.864, 1.64, 0.322, 0.259, 0.296, 0.678, ... 0.554, 0.682, 3.38, 0.0898, 0.182] [0.863, 1.64, 0.323, 0.255, 0.295, 0.676, ... 0.551, 0.678, 3.32, 0.0863, 0.183]
5 length doesnt match 14 17
6 length doesnt match 14 18
7 ptrel doesnt match [0.618, 1.49, 0.467, 1.56, 0.43, 3.48, 0.843, ... 3.39, 1.68, 1.56, 1.4, 1.25, 1.51] [0.615, 1.47, 0.465, 1.55, 0.427, 3.46, 0.84, ... 3.38, 1.68, 1.56, 

38 ptrel doesnt match [0.698, 0.227, 0.416, 0.838, 0.751, 0.505, 0.874, 0.642, 0.591] [0.671, 0.23, 0.416, 0.837, 0.755, 0.522, 0.88, 0.642, 0.592]
39 ptrel doesnt match [1, 1.33, 1.62, 0.768, 3.01, 0.934, 5.16, ... 0.732, 1.07, 6.31, 4.05, 2.32, 0.524] [1, 1.33, 1.62, 0.768, 3.01, 0.934, 5.14, ... 0.732, 1.07, 6.31, 4.05, 2.32, 0.523]
40 length doesnt match 19 20
41 ptrel doesnt match [1.21, 0.555, 0.73, 0.985, 0.913, 0.591, ... 0.265, 0.855, 0.427, 0.6, 0.581, 0.198] [1.21, 0.555, 0.731, 0.983, 0.912, 0.59, ... 0.864, 0.425, 0.599, 0.582, 0.203]
42 length doesnt match 15 16
43 ptrel doesnt match [0.476, 0.329, 0.565, 0.482, 0.689, 0.46, ... 0.384, 0.663, 0.189, 0.936, 0.618] [0.479, 0.323, 0.556, 0.466, 0.7, 0.447, ... 0.383, 0.666, 0.187, 0.935, 0.625]
44 length doesnt match 16 19
45 pt doesnt match [44, 26.9, 22.5, 6.08, 10.7, 8.88, 10.3, ... 16.8, 6.17, 5.36, 94.4, 7.29, 10, 5.05] [44, 26.9, 22.5, 6.08, 10.7, 8.88, 10.3, ... 70.5, 16.8, 6.17, 94.4, 7.29, 10, 5.05]
45 eta doesnt ma

<Array [0.0429, 0.0429, 0.0429, ... 132, 150] type='76 * float32'>

In [10]:
ievt = 54
for i in range(len(events.GenPart[ievt].pt)):
    print(
        i,
        "\t",
        events.GenPart[ievt].pdgId[i],
        "\t",
        events.GenPart[ievt].pt[i],
        "\t",
        events.GenPart[ievt].status[i],
        "\t",
        bool((events.GenPart[ievt].statusFlags & (1 << 13))[i]),
        "\t",
        events.GenPart[ievt].genPartIdxMother[i],
        events.GenPart[ievt].genPartIdxMother2[i],
        np.hypot(events.GenPart[ievt].vx[i], events.GenPart[ievt].vy[i]),
    )

0 	 2 	 0.0 	 21 	 False 	 -1 -1 0.04285188611507102
1 	 21 	 0.0 	 21 	 False 	 -1 -1 0.04285188611507102
2 	 2 	 540.0 	 23 	 False 	 0 1 0.04285188611507102
3 	 21 	 540.0 	 23 	 False 	 0 1 0.04285188611507102
4 	 2 	 594.0 	 44 	 False 	 2 -1 0.04285188611507102
5 	 21 	 24.1875 	 51 	 True 	 3 -1 0.04285188611507102
6 	 21 	 1.27734375 	 62 	 True 	 3 -1 0.04285188611507102
7 	 211 	 0.102294921875 	 2 	 True 	 -1 -1 0.04285188611507102
8 	 2 	 23.0 	 71 	 True 	 5 -1 0.04285188611507102
9 	 21 	 3.46875 	 71 	 True 	 3 -1 0.04285188611507102
10 	 2 	 243.5 	 71 	 True 	 4 -1 0.04285188611507102
11 	 21 	 24.75 	 71 	 True 	 4 -1 0.04285188611507102
12 	 21 	 202.0 	 71 	 True 	 4 -1 0.04285188611507102
13 	 21 	 19.9375 	 71 	 True 	 4 -1 0.04285188611507102
14 	 -3 	 31.875 	 71 	 True 	 4 -1 0.04285188611507102
15 	 -211 	 0.541015625 	 2 	 True 	 -1 -1 0.04285188611507102
16 	 3 	 9.9375 	 71 	 True 	 4 -1 0.04285188611507102
17 	 21 	 8.6875 	 71 	 True 	 4 -1 0.042851886115

 	 3 -1 0.04285188611507102
23 	 21 	 5.765625 	 71 	 True 	 3 -1 0.04285188611507102
24 	 21 	 0.564453125 	 71 	 True 	 3 -1 0.04285188611507102
25 	 21 	 10.1875 	 71 	 True 	 3 -1 0.04285188611507102
26 	 21 	 15.8125 	 71 	 True 	 3 -1 0.04285188611507102
27 	 21 	 75.25 	 71 	 True 	 3 -1 0.04285188611507102
28 	 21 	 3.359375 	 71 	 True 	 3 -1 0.04285188611507102
29 	 21 	 8.75 	 71 	 True 	 3 -1 0.04285188611507102
30 	 21 	 3.2421875 	 71 	 True 	 3 -1 0.04285188611507102
31 	 21 	 12.09375 	 71 	 True 	 3 -1 0.04285188611507102
32 	 21 	 8.0625 	 71 	 True 	 3 -1 0.04285188611507102
33 	 21 	 5.296875 	 71 	 True 	 3 -1 0.04285188611507102
34 	 21 	 5.703125 	 71 	 True 	 3 -1 0.04285188611507102
35 	 21 	 2.1875 	 71 	 True 	 3 -1 0.04285188611507102
36 	 21 	 68.75 	 71 	 True 	 3 -1 0.04285188611507102
37 	 21 	 246.5 	 71 	 True 	 3 -1 0.04285188611507102
38 	 21 	 16.625 	 71 	 True 	 3 -1 0.04285188611507102
39 	 21 	 9.96875 	 71 	 True 	 3 -1 0.04285188611507102
40 	

In [ ]:
1 << 0

In [4]:
ak.num(events.LostTrk)

<Array [3, 22, 21, 12, 38, ... 35, 27, 29, 21] type='200 * int64'>

In [9]:
print(events.JetPFCands.pf.d0 - events.JetPFCands.dxyFromPV)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... -0.00107, -0.0018, 0.0714, 0, 0, 0, 0, 0, 0]]


<Array [[-0.00424, -0.00171, ... -1, -1]] type='200 * var * float32[parameters={...'>